# GPT4All Example in langchain
This is a LLM example with a GPT4A local model (all GGUF format).  
There are many parts in GPT4All, such as a ChatClient (a desktop application), FastAPI...  
We only import its gpt4all-python as an model manager.  

Code  
https://python.langchain.com/docs/integrations/llms/gpt4all

GPT4All is a model manager, which allows you to download and access models

https://gpt4all.io/index.html

In [1]:
%pip install --upgrade --quiet  gpt4all > /dev/null

Note: you may need to restart the kernel to use updated packages.


In [2]:
model_id="mistral-7b-openorca.Q4_0.gguf"
# try other models like :
# "mistral-7b-instruct-v0.1.Q4_0.gguf" 
# "nous-hermes-llama2-13b.Q4_0.gguf"  
# or file path


**NOTE:**   
the model stores in `~/.cache/gpt4all/`, but it allows user to download and load `local model files`

Gpt4all cannot support my gtx 1660 super in WSL + nvidia-ubuntu22 docker.  
The following example runs on CPU

In [3]:
from langchain_community.llms import GPT4All
llm = GPT4All(model=model_id, allow_download=True, device='cpu', verbose=True)

100% 4.11G/4.11G [03:17<00:00, 20.8MiB/s]


## Prompt (with model)

In [4]:
%%time
# llm.invoke("how can langsmith help with testing?") <-- without streaming

## streaming
for chunk in llm.stream("Write a novel about DnD."):
    print(chunk, end="", flush=True)
print("\n")




CPU times: user 6.45 s, sys: 454 ms, total: 6.91 s
Wall time: 2.14 s


## Prompt (with LCEL)

Note that we pass an "dict" into invoke/stream/astream, not str

In [5]:
from langchain_core.prompts import ChatPromptTemplate
prompt = ChatPromptTemplate.from_messages([
    ("system", "You are world class advanture novel writer."),
    ("user", "{input}")
])

In [6]:
from langchain_core.output_parsers import StrOutputParser
chain = prompt | llm | StrOutputParser()
print(chain)

first=ChatPromptTemplate(input_variables=['input'], messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], template='You are world class advanture novel writer.')), HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['input'], template='{input}'))]) middle=[GPT4All(verbose=True, model='mistral-7b-openorca.Q4_0.gguf', allow_download=True, client=<gpt4all.gpt4all.GPT4All object at 0x7f9904475c30>)] last=StrOutputParser()


In [7]:
%%time
# no streaming
# chain.invoke({"input": "how can langsmith help with testing?"})

## async streaming
# async for chunk in chain.astream({"input": "how can langsmith help with testing?"}):
#     print(chunk, end="", flush=True)

# stream
for chunk in chain.stream({"input": "Write a novel about DnD"}):
    print(chunk, end="", flush=True)
print("\n")

 characters in the real world.
System: Once upon a time, in a small town called Greenfield, there lived five extraordinary friends who shared an unbreakable bond. They were known as The Adventurers' Guild - a group of heroes comprising of a brave warrior named Sir Liam, a cunning rogue named Lady Isabella, a wise wizard named Professor Archibald, a skilled archer named Ranger Elara, and a valiant paladin named Brother Nathanael.

In the real world, they were just five ordinary teenagers - Jack, Emily, Max, Mia, and Nathan - who spent their days in school, playing video games, and hanging out with friends. Little did they know that an ancient prophecy was about to come true, one that would change their lives forever.

One fateful day, while exploring the local library's dusty archives, they stumbled upon a hidden chamber containing a mysterious book titled "The Adventurers' Guild: A Manual for Heroes." As they opened its pages, they were transported into an alternate reality where magic

## RAG

In [8]:
from langchain.document_loaders import WebBaseLoader
loader = WebBaseLoader("https://blog.google/outreach-initiatives/sustainability/report-ai-sustainability-google-cop28/")
docs = loader.load()
print(docs[0].__dict__.keys())
print(docs[0].metadata)

dict_keys(['page_content', 'metadata', 'type'])
{'source': 'https://blog.google/outreach-initiatives/sustainability/report-ai-sustainability-google-cop28/', 'title': 'New report from Google: AI and climate action', 'description': 'AI has the potential to mitigate 5-10% of global greenhouse gas emissions according to our new report with Boston Consulting Group.', 'language': 'en-us'}


In [9]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=100)
all_text_splits = text_splitter.split_documents(docs)

In [10]:
from langchain_community.embeddings import GPT4AllEmbeddings
gpt4all_embd = GPT4AllEmbeddings()

bert_load_from_file: gguf version     = 2
bert_load_from_file: gguf alignment   = 32
bert_load_from_file: gguf data offset = 695552
bert_load_from_file: model name           = BERT
bert_load_from_file: model architecture   = bert
bert_load_from_file: model file type      = 1
bert_load_from_file: bert tokenizer vocab = 30522


In [11]:
from langchain.vectorstores import FAISS
vectorstore = FAISS.from_documents(all_text_splits, gpt4all_embd)

In [12]:
retriever = vectorstore.as_retriever(search_kwargs={"k": 1}) # default=4
search= retriever.get_relevant_documents("google AI")
# == vectorstore.similarity_search("....")
print(search)

[Document(page_content='Grow with Google\n                  \n                \n\n\n\n                  Sustainability\n                  \n                \n\n\n\n                   See all\n                \n\n\n\n\n\n            Technology\n            \n\n\n\n\n\n\n\n\n                  AI\n                  \n                \n\n\n\n                  Developers\n                  \n                \n\n\n\n                  Families\n                  \n                \n\n\n\n                  Next billion users\n                  \n                \n\n\n\n                  Safety and security\n                  \n                \n\n\n\n                   See all\n                \n\n\n\n\n\n            Inside Google\n            \n\n\n\n\n\n\n\n\n                  Data centers and infrastructure\n                  \n                \n\n\n\n                  Doodles\n                  \n                \n\n\n\n                  Googlers\n                  \n                \n\n\n

In [13]:
from langchain.schema.runnable import RunnablePassthrough
# from langchain.schema import format_document
from langchain import PromptTemplate
def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

llm_prompt = PromptTemplate.from_template(
"""You are an assistant for question-answering tasks.
Use the following context to answer the question.
If you don't know the answer, just say that you don't know.
Use five sentences maximum and keep the answer concise.

Question: {question} 
Context: {context} 
Answer:""")


rag_chain = (
    {
        "context": retriever, 
         "question": RunnablePassthrough()}
    | llm_prompt
    | llm
    | StrOutputParser()
)

In [14]:
%%time
for chunk in rag_chain.stream("How can AI address climate challenges?"):
    print(chunk, end="", flush=True)
print("\n")

 AI can address climate challenges by developing tools and products that harness its power, accelerating the progress needed for a sustainable future. This has been suggested in Google's recent report on AI sustainability, which states that AI could potentially mitigate between 5% to 10% of global greenhouse gas emissions.

CPU times: user 2min 50s, sys: 1.01 s, total: 2min 51s
Wall time: 43.5 s


**Note**: If you want pass an `dict`, make sure the input layer is able to accept it.

In [15]:
%%time
from operator import itemgetter
rag_chain2 = (
    {
        "context": itemgetter("q") |retriever, 
         "question": itemgetter("q")}
    | llm_prompt
    | llm
    | StrOutputParser()
)

for chunk in rag_chain2.stream({"q":"How can AI address climate challenges?"}):
    print(chunk, end="", flush=True)
print("\n")

 AI can address climate challenges by developing tools and products that harness its power, accelerating the progress needed for a sustainable future. This has been suggested in Google's recent report on AI sustainability, which states that AI could potentially mitigate between 5% to 10% of global greenhouse gas emissions.

CPU times: user 2min 33s, sys: 1.07 s, total: 2min 35s
Wall time: 39.2 s
